<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [11]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [12]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [13]:
%sql sqlite:///my_data1.db

In [14]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [ ]:
%%sql
DROP TABLE IF EXISTS SPACEXTABLE;

In [ ]:
%%sql
create table SPACEXTABLE as 
    select * from SPACEXTBL where Date is not null;

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [15]:
%%sql
SELECT DISTINCT Launch_Site FROM SPACEXTABLE;

,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [16]:
%%sql
SELECT * 
FROM SPACEXTABLE WHERE 
    Launch_Site LIKE 'CCA%' 
LIMIT 5;

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [17]:
%%sql
SELECT 
    SUM(PAYLOAD_MASS__KG_) 
FROM SPACEXTABLE WHERE 
    Customer LIKE 'NASA%';

,SUM(PAYLOAD_MASS__KG_)
0,99980


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [18]:
%%sql
SELECT 
    AVG(PAYLOAD_MASS__KG_) 
FROM SPACEXTABLE WHERE
    Booster_Version LIKE 'F9 v1.1%';

,AVG(PAYLOAD_MASS__KG_)
0,2534.666667


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [19]:
%%sql
SELECT 
    MIN(Date) 
FROM SPACEXTABLE WHERE 
    Landing_Outcome LIKE 'Success (ground pad)';

,MIN(Date)
0,2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [20]:
%%sql
SELECT 
    Booster_Version 
FROM SPACEXTABLE WHERE 
    Landing_Outcome LIKE 'Success (drone ship)' AND 
    PAYLOAD_MASS__KG_ > 4000 AND 
    PAYLOAD_MASS__KG_ < 6000;

,Booster_Version
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


### Task 7




##### List the total number of successful and failure mission outcomes


In [21]:
%%sql
SELECT 
    Mission_Outcome,
    COUNT(*) AS value 
FROM SPACEXTABLE 
GROUP BY Mission_Outcome;

,Mission_Outcome,value
0,Failure (in flight),1
1,Success,98
2,Success,1
3,Success (payload status unclear),1


In [22]:
%%sql
SELECT
    SUM(CASE WHEN Mission_Outcome LIKE 'Success%' THEN 1 ELSE 0 END) AS Nb_Success,
    SUM(CASE WHEN Mission_Outcome LIKE 'Failure%' THEN 1 ELSE 0 END) AS Nb_Failure
FROM SPACEXTABLE;

,Nb_Success,Nb_Failure
0,100,1


In [23]:
%%sql
SELECT 
    count(Mission_Outcome) 
FROM SPACEXTABLE WHERE 
    Mission_Outcome LIKE 'Success%';

,count(Mission_Outcome)
0,100


In [24]:
%%sql
SELECT 
    count(Mission_Outcome) 
FROM SPACEXTABLE WHERE 
    Mission_Outcome LIKE 'Failure%';

,count(Mission_Outcome)
0,1


### Task 8



##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [25]:
%%sql
SELECT 
    Booster_Version 
FROM SPACEXTABLE WHERE 
    PAYLOAD_MASS__KG_ = (
        SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE
    );

,Booster_Version
0,F9 B5 B1048.4
1,F9 B5 B1049.4
2,F9 B5 B1051.3
3,F9 B5 B1056.4
4,F9 B5 B1048.5
5,F9 B5 B1051.4
6,F9 B5 B1049.5
7,F9 B5 B1060.2
8,F9 B5 B1058.3
9,F9 B5 B1051.6


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [26]:
%%sql
SELECT 
    substr(Date, 6,2) as month, 
    Landing_Outcome, 
    Booster_Version, 
    Launch_Site 
FROM SPACEXTABLE WHERE 
    substr(Date,0,5)='2015' AND 
    Landing_Outcome LIKE 'Failure (drone ship)';

,month,Landing_Outcome,Booster_Version,Launch_Site
0,1,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
1,4,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [27]:
%%sql
SELECT 
    Landing_Outcome, 
    COUNT(*) value 
FROM SPACEXTABLE WHERE 
    Date BETWEEN '2010-06-04' AND '2017-03-20' 
GROUP BY "Landing_Outcome" ORDER BY value DESC;

,Landing_Outcome,value
0,No attempt,10
1,Success (drone ship),5
2,Failure (drone ship),5
3,Success (ground pad),3
4,Controlled (ocean),3
5,Uncontrolled (ocean),2
6,Failure (parachute),2
7,Precluded (drone ship),1


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
